In [35]:
import torch
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from celluloid import Camera
from IPython.display import HTML
import time

In [41]:
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
import torch
import lightning.pytorch as pl

from model.baseline_unet import UNet
from model.attention_unet import A_UNET
from model.dilated_unet import DilatedUNet
from model.resnet_unet import ResNetUNet
from model.baseline_lightning import LightningSegmentation

from eval.loss import BCEWithLogitAndDice, DiceLoss, DiceScore
from eval.eval_utils import get_dice_score

In [4]:
%load_ext autoreload
%autoreload 2

In [12]:
from visualization import make_mri_gif

In [5]:
mri = nib.load("/home/nguye/cse493/LungTumor-Segmentation-main/dataset/Task06_Lung/imagesTs/lung_002.nii.gz")

In [17]:
file_data = mri.get_fdata()
file_data.shape
slice = np.expand_dims(file_data[:,:,100], axis=0)
slice = np.expand_dims(slice, axis=0)
slice.shape

(1, 1, 512, 512)

In [7]:
model = LightningSegmentation.load_from_checkpoint("/home/nguye/cse493/segementation/lightning_logs/dilated_unet/version_2/checkpoints/epoch=12-step=3874.ckpt")

new change


/opt/conda/lib/python3.7/site-packages/lightning/pytorch/utilities/parsing.py:270: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."
/opt/conda/lib/python3.7/site-packages/lightning/pytorch/utilities/parsing.py:270: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."


In [29]:
model.eval()
model.freeze()
out = model(torch.Tensor(slice))

In [30]:
process_mask = torch.sigmoid(out).cpu().detach().numpy().copy()

In [32]:
process_mask = np.where(process_mask > 0.5, 1, 0)

In [34]:
np.unique(process_mask)

array([0, 1])

In [63]:
def model_loader_inference(model_path, model_obj, scan_test_file, thresh_hold=0.5, num_result=10):
    model = LightningSegmentation.load_from_checkpoint(model_path)
    mri_file = nib.load(scan_test_file)
    file_data = mri_file.get_fdata()
    
    model.eval()
    model.freeze()
    result = []
    for i in range(num_result):
      start_time = time.time()
      for i in range(file_data.shape[2]):
          slice = np.expand_dims(file_data[:,:,i], axis=0)
          slice = np.expand_dims(slice, axis=0)
          out = model(torch.Tensor(slice))
          process_mask = torch.sigmoid(out).cpu().detach().numpy().copy()
          process_mask = np.where(process_mask > thresh_hold, 1, 0)
          # file_data[:,:,i] = process_mask[0,0,:,:]
      result_time = time.time() - start_time
      print("--- %s seconds ---" % (result_time))
      result.append(result_time)
    
    mean = np.array(result).mean()
    sd = np.array(result).std()
    print(f"Time record: {mean} +- {sd}")

    # fig = plt.figure()
    # camera = Camera(fig)  # create the camera object from celluloid

    # for i in range(0, ct.shape[2], 2):  # axial view
    #     plt.imshow(ct[:,:,i], cmap="bone")
    #     mask_ = np.ma.masked_where(mask[:,:,i]==0, mask[:,:,i])
    #     plt.imshow(mask_, alpha=0.5, cmap="autumn")
    #     #plt.axis("off")
    #   camera.snap()  # Store the current slice
    # animation = camera.animate() 



In [66]:
model_checkpoint = "/home/nguye/cse493/segementation/lightning_logs/dilated_unet/version_2/checkpoints/epoch=12-step=3874.ckpt"
# model_checkpoint = "/home/nguye/cse493/segementation/lightning_logs/unet_adam/version_9/checkpoints/epoch=16-step=2533.ckpt"
# model_checkpoint = "/home/nguye/cse493/segementation/lightning_logs/resnet_unet/version_9/checkpoints/epoch=35-step=57996.ckpt"
# model_checkpoint ="/home/nguye/cse493/segementation/lightning_logs/resnet_unet/version_4/checkpoints/epoch=24-step=7450.ckpt"
model_obj = DilatedUNet()
scan_test_file = "/home/nguye/cse493/LungTumor-Segmentation-main/dataset/Task06_Lung/imagesTs/lung_002.nii.gz"

model_loader_inference(model_checkpoint, model_obj, scan_test_file)

new change
--- 13.62182068824768 seconds ---
--- 13.75895619392395 seconds ---
--- 13.940370321273804 seconds ---
--- 13.767361402511597 seconds ---
--- 13.641370296478271 seconds ---
--- 13.589959621429443 seconds ---
--- 13.578168869018555 seconds ---
--- 13.631057739257812 seconds ---
--- 13.671933650970459 seconds ---
--- 13.699512720108032 seconds ---
Time record: 13.69005115032196 +- 0.1033651951188134


In [16]:
a_unet = A_UNET(dropout=0)
model_path = "/home/nguye/cse493/segementation/lightning_logs/unet_attention/version_8"
mri_path = "/home/nguye/cse493/LungTumor-Segmentation-main/dataset/Task06_Lung/imagesTs/lung_002.nii.gz"
video = make_mri_gif(a_unet, model_path, mri_path)
display(video)

IsADirectoryError: [Errno 21] Is a directory: '/home/nguye/cse493/segementation/lightning_logs/unet_attention/version_8'